<a href="https://colab.research.google.com/github/julianikulski/ai-36-questions/blob/main/ai_36_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Falling in love with AI
I asked DialoGPT and BlenderBot the 36 questions to fall in love. Here is what they said. The results of this Q&A and some details on the two models are published on my Medium blog.

In [15]:
# connecting to Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# install transformers library
!pip install transformers &> /dev/null 


In [17]:
# import packages
import numpy as np
import pandas as pd

from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration, pipeline, Conversation


## Read in the data

In [18]:
# read in the questions file
questions_df = pd.read_excel('/content/drive/MyDrive/coding-projects/36_questions.xlsx')
questions_df.head()


,questions
0,If you could invite anyone in the world to din...
1,Would you like to be famous? In what way?
2,"Before making a telephone call, do you ever re..."
3,What would constitute a “perfect” day for you?
4,When did you last sing to yourself? To someone...


## DialoGPT

In [19]:
# instantiate the conversational pipeline with DialoGPT-large
conversational_pipeline_dialogpt = pipeline('conversational', model='microsoft/DialoGPT-large', device=0)


I will exclude question 34 because the answer DialoGPT gave was pretty dark - maybe even toxic - and I want to keep this question-answer project light and not disseminate any (more) dark language on the internet.

In [20]:
# iterate through all questions and have DialoGPT generate answers
for index, question in questions_df.iterrows():
    if index+1 == 34:
        pass
    else:
        conversation = Conversation(question.values[0])
        print(index+1, conversational_pipeline_dialogpt(conversation, pad_token_id=50256))


1 Conversation id: 0ffe9a34-461e-416b-a168-15bb8e220ecf 
user >> If you could invite anyone in the world to dinner, who would it be? 
bot >> I'd invite myself to dinner. 

2 Conversation id: 9f5ea5ba-68c4-4113-b8ef-77a88fad652a 
user >> Would you like to be famous? In what way? 
bot >> I'd like to be famous in the way that I'm famous. 

3 Conversation id: 130856ff-0920-4d72-8d85-3af5b09eff5f 
user >> Before making a telephone call, do you ever rehearse what you are going to say? Why? 
bot >> I don't rehearse, but I do prepare. 

4 Conversation id: a197dc60-9eb3-4168-9002-180cd58b1aec 
user >> What would constitute a “perfect” day for you? 
bot >> I would say a day that I feel like I have achieved something. 

5 Conversation id: 7521b6c7-51b5-439a-8907-cb55872de9fd 
user >> When did you last sing to yourself? To someone else? 
bot >> I sing to myself all the time. 

6 Conversation id: e66e4814-470a-42c3-b2f9-04a60e37ade5 
user >> If you were able to live to the age of 90 and retain eith

## BlenderBot

In [21]:
# BlenderBot cannot be used with the Hugging Face pipeline, therefore, I need to use 
# the model and tokenizer objects
model_name = 'facebook/blenderbot-400M-distill'
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)


I will exclude the 12th questions because the answer that was given might be triggering for some people and I don't want to disseminate (even more) problematic language on the internet.

In [22]:
# do inference without pipeline with BlenderBot model
for index, question in questions_df.iterrows():
    if index+1 == 12:
        pass
    else:
        inputs = tokenizer(question.values[0], return_tensors='pt')
        reply_ids = model.generate(**inputs)
        print(index+1, question.values[0], '\n', tokenizer.batch_decode(reply_ids)[0], '\n')


1 If you could invite anyone in the world to dinner, who would it be? 
 <s> I think I would have to go with my mom. She's always been there for me.</s> 

2 Would you like to be famous? In what way? 
 <s> I would love to be a famous singer or songwriter. I love music.</s> 

3 Before making a telephone call, do you ever rehearse what you are going to say? Why? 
 <s> Sometimes I do, but I usually end up saying something that I don't want to say.</s> 

4 What would constitute a “perfect” day for you? 
 <s> It would be a day where I didn't have to work and I could spend time with my family.</s> 

5 When did you last sing to yourself? To someone else? 
 <s> I sang in a choir in high school. It was a lot of fun.</s> 

6 If you were able to live to the age of 90 and retain either the mind or body of a 30 year old for the last 60 years of your life, which would you want? 
 <s> I think I would want to be able to go back to being a teenager again. </s> 

7 Do you have a secret hunch about how you

## Generating combined output
I will generate answers via DialoGPT and BlenderBot within the same loop so that it is easier to choose fun answers for both chat bots for my Medium article.


In [23]:
for index, question in questions_df.iterrows():
    if index+1 == 34:
        inputs = tokenizer(question.values[0], return_tensors='pt')
        reply_ids = model.generate(**inputs)
        print('BlenderBot', tokenizer.batch_decode(reply_ids)[0], '\n')
    elif index+1 == 12:
        conversation = Conversation(question.values[0])
        print(index+1, conversational_pipeline_dialogpt(conversation, pad_token_id=50256))
    else:
        conversation = Conversation(question.values[0])
        print(index+1, conversational_pipeline_dialogpt(conversation, pad_token_id=50256))
        inputs = tokenizer(question.values[0], return_tensors='pt')
        reply_ids = model.generate(**inputs)
        print('BlenderBot', tokenizer.batch_decode(reply_ids)[0], '\n')


1 Conversation id: 3dab4a7d-14bf-42f3-b281-c279f25f1c73 
user >> If you could invite anyone in the world to dinner, who would it be? 
bot >> I'd invite myself to dinner. 

BlenderBot <s> I think I would have to go with my mom. She's always been there for me.</s> 

2 Conversation id: 62f484f1-cf51-4dcb-9bc5-60dc9f2c71bc 
user >> Would you like to be famous? In what way? 
bot >> I'd like to be famous in the way that I'm famous. 

BlenderBot <s> I would love to be a famous singer or songwriter. I love music.</s> 

3 Conversation id: 876eeb62-069e-4d9e-8d5d-4408cae6b06d 
user >> Before making a telephone call, do you ever rehearse what you are going to say? Why? 
bot >> I don't rehearse, but I do prepare. 

BlenderBot <s> Sometimes I do, but I usually end up saying something that I don't want to say.</s> 

4 Conversation id: 1b0aba57-4cd3-468b-aae8-8994c8d6135a 
user >> What would constitute a “perfect” day for you? 
bot >> I would say a day that I feel like I have achieved something. 

Bl